# Load, preprocess, and save train and test data

This notebook preprocesses and collates the training and testing data for model creation.

# John Brandt
# July 11, 2021

- Fuse Sentinel 1/2 data
- Reconstruct 2D-array from CEO output CSV by plot
- Match sentinel data to CEO labels
- Stack data_x, data_y, length
- Save arrays for data_x, data_y, length


# Package imports and source code

In [1]:
from tqdm import tqdm_notebook, tnrange
import pandas as pd
import numpy as np
from random import shuffle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import random
import itertools
from scipy.ndimage import median_filter
import hickle as hkl

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%run ../src/preprocessing/slope.py

In [2]:
def reconstruct_images(plot_id):
    '''Takes a plot ID and subsets the input pd.DataFrame to that plot ID
       returns a (14, 14) array-like list with binary labels
       
        Parameters:
          batch_ids (list):
          batch_size (int):
          
         Returns:
          x_batch (arr):
          y_batch (arr):
    '''
    subs = df[df['PLOT_ID'] == plot_id]
    rows = []
    lats = reversed(sorted(subs['LAT'].unique()))
    for i, val in enumerate(lats):
        subs_lat = subs[subs['LAT'] == val]
        subs_lat = subs_lat.sort_values('LON', axis = 0)
        rows.append(list(subs_lat['TREE']))
    return rows

In [3]:
source = 'train'
sentinel_1 = True
s2_path = "../data/{}-s2-new/".format(source)
s1_path = "../data/{}-s1-radiometric-32/".format(source)
csv_path = "../data/{}-csv/".format(source)
output_path = "../data/{}-processed/".format(source)
dem_path = "../data/{}-dem/".format(source)

In [4]:
# Load and edit bad plot ids if needed
verified_lu_change = np.load("bad_plot_ids.npy")
len(verified_lu_change)

to_add = [141238348]
to_add = [x for x in to_add if x not in verified_lu_change]
verified_lu_change = np.concatenate([verified_lu_change, 
                     np.array(to_add).flatten()])

to_remove = []

verified_lu_change = [x for x in verified_lu_change if x not in to_remove]
np.save("bad_plot_ids.npy", np.array(verified_lu_change))
print(len(verified_lu_change))

2328


In [5]:
# For either train or test data, loop through each plot and determine whether there is
# labelled Y data for it -- returning one dataframe for the entire data set

cols_to_keep = ['PLOT_ID', 'SAMPLE_ID', 'LON', 'LAT', 'FLAGGED', 'ANALYSES', 'USER_ID',
       'COLLECTION_TIME', 'ANALYSIS_DURATION', 'TREE', 'plotid', 'sampleid']
csvs = [x for x in sorted(os.listdir(csv_path)) if ".csv" in x]
#csvs = [x for x in csvs if 'uuid'in x]
csvs = [x for x in csvs if ".csv" in x]
csvs = [x for x in csvs if "kenya" not in x]
csvs = [x for x in csvs if "senegal" not in x]



dfs = []
for i in csvs:
    df = pd.read_csv(csv_path + i, encoding = "ISO-8859-1")
    df.columns = [x.upper() for x in df.columns]
    df.columns = ['PLOT_ID' if x == 'PLOTID' else x for x in df.columns]
    df.columns = ['SAMPLE_ID' if x == 'SAMPLEID' else x for x in df.columns]
    
    print(df['PLOT_ID'][0])
    
    # If there are no unique IDs already, go ahead and assign them
    """
    if abs(df['PLOT_ID'][0]) == 1:
        print(df['PLOT_ID'][0])
        print(f"No unique ID for {i}")
        for index, row in df.iterrows():
            row['PLOT_ID'] = abs(row['PLOT_ID'])
            df['PLOT_ID'][index] = int(str(i[-5:-4]) + '00' + str(row['PLOT_ID']))
    """
    print(df['PLOT_ID'].unique())
    
    #print(df.columns)

    for column in df.columns:
        if column not in cols_to_keep:
            df = df.drop(column, axis = 1)
            
    df['country'] = i.split(".")[0]
    #print(( len(df) - np.sum(pd.isna(df['TREE']))) / 196 )
    print(len(df.columns))
    #df.to_csv(csv_path + i, index = False)
    dfs.append(df)

df = pd.concat(dfs, ignore_index = True, sort = True)
print(len(df) // 196)
df = df[~pd.isna(df['TREE'])]
print(len(df) // 196)

plot_ids = sorted(df['PLOT_ID'].unique())
plot_ids_loaded = plot_ids

print(f"There are {len(plot_ids)} plots")

141237861
[141237861 141237862 141237863 141237864 141237865 141237866 141237867
 141237868 141237869 141237870 141237871 141237872 141237873 141237874
 141237875 141237876 141237877 141237878 141237879 141237880 141237881
 141237882 141237883 141237884 141237885 141237886 141237887 141237888
 141237889 141237890 141237891 141237892 141237893 141237894 141237895
 141237896 141237897 141237898 141237899 141237900 141237901 141237902
 141237903 141237904 141237905 141237906 141237907 141237908 141237909
 141237910 141237911 141237912 141237913 141237914 141237915 141237916
 141237917 141237918 141237919 141237920 141237921 141237922 141237923]
9
1
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96
 97 98]
9
1001
[  1001   1002   1003   1004   1005   1

6001
[  6001   6002   6003   6004   6005   6006   6007   6008   6009  60010
  60011  60012  60013  60014  60015  60016  60017  60018  60019  60020
  60021  60022  60023  60024  60025  60026  60027  60028  60029  60030
  60031  60032  60033  60034  60035  60036  60037  60038  60039  60040
  60041  60042  60043  60044  60045  60046  60047  60048  60049  60050
  60051  60052  60053  60054  60055  60056  60057  60058  60059  60060
  60061  60062  60063  60064  60065  60066  60067  60068  60069  60070
  60071  60072  60073  60074  60075  60076  60077  60078  60079  60080
  60081  60082  60083  60084  60085  60086  60087  60088  60089  60090
  60091  60092  60093  60094  60095  60096  60097  60098  60099 600100
 600101 600102 600103 600104 600105 600106 600107 600108 600109 600110
 600111 600112 600113 600114 600115 600116 600117 600118 600119 600120
 600121 600122 600123 600124 600125 600126 600127 600128 600129 600130
 600131 600132 600133 600134 600135 600136 600137 600138 600139 600140
 

[141019141 141019142 141019143 141019144 141019145 141019146 141019147
 141019148 141019149 141019150 141019151 141019152 141019153 141019154
 141019155 141019156 141019157 141019158 141019159 141019160 141019161
 141019162 141019163 141019164 141019165 141019166 141019167 141019168
 141019169 141019170 141019171 141019172 141019173 141019174 141019175
 141019176 141019177 141019178 141019179 141019180 141019181 141019182
 141019183 141019184 141019185 141019186 141019187 141019188 141019189
 141019190 141019191 141019192 141019193 141019194 141019195 141019196
 141019197 141019198 141019199 141019200 141019201 141019202 141019203
 141019204 141019205 141019206 141019207 141019208 141019209 141019210
 141019211 141019212 141019213 141019214 141019215 141019216 141019217
 141019218 141019219 141019220 141019221 141019222 141019223 141019224
 141019225 141019226 141019227 141019228 141019229 141019230 141019231
 141019232 141019233 141019234 141019235 141019236 141019237 141019238
 14101

In [8]:
df = df.to_csv("15_percent_sample.csv")

In [6]:
def to_int16(array: np.array) -> np.array:
    '''Converts a float32 array to int16, reducing storage costs by three-fold'''
    assert np.min(array) >= 0, np.min(array)
    assert np.max(array) <= 1, np.max(array)
    
    array = np.clip(array, 0, 1)
    array = np.trunc(array * 65535)
    assert np.min(array >= 0)
    assert np.max(array <= 65535)
    
    return array.astype(np.uint16)

def process_dem(dem):
    dem =  median_filter(dem, size = 5)
    dem = calcSlope(dem.reshape((1, 32+2, 32+2)),
                      np.full((32+2, 32+2), 10),
                      np.full((32+2, 32+2), 10), 
                      zScale = 1, minSlope = 0.02)
    dem = dem / 90
    dem = dem.reshape((32+2, 32+2, 1))
    dem = dem[1:-1, 1:-1]
    dem = median_filter(dem, 5)[2:-2, 2:-2]
    return dem

def grndvi(array):
    nir = np.clip(array[..., 3], 0, 1)
    green = np.clip(array[..., 1], 0, 1)
    red = np.clip(array[..., 2], 0, 1)
    denominator = (nir+(green+red)) + 1e-5
    return (nir-(green+red)) / denominator


In [ ]:
from skimage.transform import resize

%run ../src/preprocessing/indices.py

def to_float32(array: np.array) -> np.array:
    """Converts an int_x array to float32"""
    if not isinstance(array.flat[0], np.floating):
        assert np.max(array) > 1
        array = np.float32(array) / 65535.
    assert np.max(array) <= 1
    assert array.dtype == np.float32
    return array

count = 0
dataframe = pd.DataFrame({'plot_id': [''], 'lat': [0.325], 'long': [0.325],
                          'y': [0]})

# Identify shape of data to load
plot_ids_to_load = []
for i in range(len(plot_ids)):
    s1_i = f'{s1_path}{str(plot_ids[i])}.hkl'
    s2_i = f'{s2_path}{str(plot_ids[i])}.hkl'
    dem_i = f'{dem_path}{str(plot_ids[i])}.npy'
    s1_new_i = f'../data/{source}-s1/{str(plot_ids[i])}.npy'
    s1_exists = (os.path.exists(s1_i))
    
    if os.path.isfile(s2_i) and s1_exists:
        if plot_ids[i] not in verified_lu_change:
            plot_ids_to_load.append(plot_ids[i])

print(f"There are {len(plot_ids_to_load)} plots")
plot_ids_to_load = [x for x in plot_ids_to_load if x not in  [139077414,
                                                              139187051,
                                                              139187043,
                                                             139187133, 139187134]]
data_x = np.zeros((len(plot_ids_to_load), 12, 28, 28, 14)).astype(np.uint16)
data_y = np.zeros((len(plot_ids_to_load), 14, 14))
            
# Iterate over each plot
to_remove = []

for i in range(len(plot_ids_to_load)):
    s1_i = f'{s1_path}{str(plot_ids_to_load[i])}.hkl'
    s2_i = f'{s2_path}{str(plot_ids_to_load[i])}.hkl'
    dem_i = f'{dem_path}{str(plot_ids_to_load[i])}.npy'

    x = to_float32(hkl.load(s2_i))
    s1 = hkl.load(s1_i)
    s1 = np.reshape(s1, (12, 16, 2, 16, 2, 2))
    s1 = np.mean(s1, axis = (2, 4))
    s1 = resize(s1, (12, 32, 32, 2), order = 1)
    s1 = s1[:, 2:-2, 2:-2, :]
    
    dem = np.load(dem_i)
    dem = process_dem(dem)
    dem = np.tile(dem.reshape((1, 28, 28)), (x.shape[0], 1, 1))
    x[..., 10] = dem
    x = np.concatenate([x, s1], axis = -1)

    count += 1
    y = reconstruct_images(plot_ids_to_load[i])
    long = np.mean(df[df['PLOT_ID'] == plot_ids_to_load[i]]['LON'])
    lat = np.mean(df[df['PLOT_ID'] == plot_ids_to_load[i]]['LAT'])
    dataframe = dataframe.append({'plot_id': str(plot_ids_to_load[i]),
                                  'lat': lat, 'long': long,
                                 'y': np.sum(np.array(y))}, 
                                 ignore_index = True)
    dataframe.append([plot_ids_to_load[i], lat, long])

    if np.sum(np.isnan(x)) > 0:
        to_remove.append(i)
    else:
        x = np.clip(x, 0, 1)
        x = to_int16(x)
        data_x[i] = x
        try:
            data_y[i] = np.array(y)
        except:
            to_remove.append(i)
            
# Remove any data samples that had missing values
if len(to_remove) > 0:
    print(f"Removing {to_remove}")
    data_x = np.delete(data_x, to_remove, 0)
    data_y = np.delete(data_y, to_remove, 0)
            
print(f"Finished loading: {data_x.shape} of {data_x.dtype} type")

In [ ]:
import hickle as hkl
dataframe = dataframe.drop(0, 0)
dataframe.reset_index(inplace = True, drop = True)
if len(to_remove) > 0:
    dataframe = dataframe.drop(to_remove, 0)
    dataframe.reset_index(inplace = True, drop = True)

print(f"Writing {source} data")
hkl.dump(data_x, f"../data/{source}/{source}_x.hkl", mode='w', compression='gzip')
hkl.dump(data_y, f"../data/{source}/{source}_y.hkl", mode='w', compression='gzip')
dataframe.to_csv(f"../data/{source}/{source}_plot_ids.csv", index = False)
print("Finished!")